In [42]:
import graphlab
import numpy as np
from math import sqrt

In [3]:
sales = graphlab.SFrame("E:\\Machine Learning\\U.W\\Regression\\kc_house_data.gl/")

This non-commercial license of GraphLab Create for academic use is assigned to lxn1021@gmail.com and will expire on November 18, 2019.


[INFO] graphlab.cython.cy_server: GraphLab Create v2.1 started. Logging: C:\Users\Xiaoning\AppData\Local\Temp\graphlab_server_1544656949.log.0


In [4]:
sales.head()

id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront
7129300520,2014-10-13 00:00:00+00:00,221900.0,3.0,1.0,1180.0,5650,1,0
6414100192,2014-12-09 00:00:00+00:00,538000.0,3.0,2.25,2570.0,7242,2,0
5631500400,2015-02-25 00:00:00+00:00,180000.0,2.0,1.0,770.0,10000,1,0
2487200875,2014-12-09 00:00:00+00:00,604000.0,4.0,3.0,1960.0,5000,1,0
1954400510,2015-02-18 00:00:00+00:00,510000.0,3.0,2.0,1680.0,8080,1,0
7237550310,2014-05-12 00:00:00+00:00,1225000.0,4.0,4.5,5420.0,101930,1,0
1321400060,2014-06-27 00:00:00+00:00,257500.0,3.0,2.25,1715.0,6819,2,0
2008000270,2015-01-15 00:00:00+00:00,291850.0,3.0,1.5,1060.0,9711,1,0
2414600126,2015-04-15 00:00:00+00:00,229500.0,3.0,1.0,1780.0,7470,1,0
3793500160,2015-03-12 00:00:00+00:00,323000.0,3.0,2.5,1890.0,6560,2,0


In [22]:
def get_numpy_data(data_sframe, features, output):
    data_sframe["constant"] = 1
    features = ["constant"] + features
    features_sframe = data_sframe[features]
    
    feature_matrix = features_sframe.to_numpy()
    
    output_sarray = data_sframe[output]
    output_array = output_sarray.to_numpy()
    
    
    return (feature_matrix, output_array)

In [37]:
def predict_output(feature_matrix, weights):
    predictions = np.dot(feature_matrix, weights)
    
    return predictions

In [41]:
def feature_derivative(errors, feature):
    derivative = np.dot(errors, feature)
    
    return derivative

In [124]:
def regression_gradient_descent(feature_matrix, output, initial_weights, step_size, tolerance):
    converged = False
    weights = np.array(initial_weights)
    
    while not converged:
        predictions = predict_output(feature_matrix, weights)
        errors = predictions - output
        
        gradient_sum_squares = 0
        
        for i in range(len(weights)):
            derivative = (-2 * np.mat(predictions) * (np.mat(output) - np.mat(predictions) * weights[i]).T)[0, 0]
            gradient_sum_squares = gradient_sum_squares + derivative ** 2
            weights[i] = weights[i] - step_size * derivative
        
        gradient_magnitude = sqrt(gradient_sum_squares)
        if gradient_magnitude < tolerance:
            converged = True
            
    
    return weights

In [125]:
train_data, test_data = sales.random_split(0.8, seed = 0)

In [126]:
simple_features = ['sqft_living']
my_output= 'price'
(simple_feature_matrix, output) = get_numpy_data(train_data, simple_features, my_output)
initial_weights = np.array([-47000., 1.])
step_size = 7e-12
tolerance = 2.5e7

In [127]:
regression_gradient_descent(simple_feature_matrix, output, initial_weights, step_size, tolerance)

c:\python27\lib\site-packages\ipykernel_launcher.py:13: RuntimeWarning: invalid value encountered in double_scalars
  del sys.path[0]


KeyboardInterrupt: 

In [122]:
converged = False
weights = np.array(initial_weights)

predictions = np.dot(simple_feature_matrix ,weights)
errors = predictions - output

gradient_sum_squares = 0


np.mat(predictions) 
derivative = (-2 * np.mat(predictions) * (np.mat(output) - np.mat(predictions) * weights[0]).T)[0, 0]

gradient_sum_squares = gradient_sum_squares + derivative ** 2

In [123]:
gradient_sum_squares

1.0875741525266723e+37